# 内部スイッチ作成

## スイッチ名とIPアドレス
内部通信用スイッチを作成してホスト側アダプターにIPアドレスを設定

In [ ]:
$switchName = 'InternalSwitch'
$ipAddress = '192.168.128.250'
$prefixLength = 24

## スイッチ作成

In [ ]:
$switchName = 'InternalSwitch'
$existing = Get-VMSwitch -SwitchType Internal | Where-Object {$_.Name -eq $switchName}
if (-not($existing)) {
    'Creating...'
    New-VMSwitch -Name $switchName -SwitchType Internal -Verbose
}
else {
    'Existing'
    $existing
}

## IPアドレス設定

In [ ]:
$adapterName = "vEthernet ($switchName)"
$adapter = Get-NetAdapter -Name $adapterName -ErrorAction SilentlyContinue
if ($adapter) {
    Get-NetIPAddress -InterfaceAlias $adapterName -AddressFamily IPv4 -ErrorAction SilentlyContinue | `
        Remove-NetIPAddress -Confirm:$false -ErrorAction SilentlyContinue -Verbose
    New-NetIPAddress -InterfaceAlias $adapterName -IPAddress $ipAddress -PrefixLength $prefixLength -Verbose
}

# SMB共有用設定

## SMB用ユーザー作成

In [ ]:
if(-not(Get-LocalUser -Name vagrant)) {
    New-LocalUser vagrant -Password (ConvertTo-SecureString vagrant -AsPlainText -Force) -PasswordNeverExpires -AccountNeverExpires -Verbose
}

## アクセス権設定

In [ ]:
$acl = Get-Acl '.'
$accessRule = New-Object System.Security.AccessControl.FileSystemAccessRule('vagrant', 'FullControl', 'ContainerInherit,ObjectInherit', 'None', 'Allow')
$acl.SetAccessRule($accessRule)
Set-Acl -Path '.' -AclObject $acl -Verbose